In [1]:
import unittest
from functools import partial

from conv import MyConvStub, MyFilterStub
from blur_kernel import get_blur_kernel

import torch
from torch import nn
import torch.nn.functional as F

kernel_size_stub_constructor = partial(MyConvStub, input_channels=128, output_channels=128, num_groups=1, stride=1, dilation=1, bias=False)
filter_size_stub_constructor = partial(MyConvStub, kernel_size=(3, 3), num_groups=1, stride=1, dilation=1, bias=False)
biased_filter_size_stub_constructor = partial(MyConvStub, kernel_size=(3, 3), num_groups=1, stride=1, dilation=1, bias=True)
stride_size_stub_constructor = partial(MyConvStub, input_channels=128, output_channels=128, kernel_size=(3, 3), num_groups=1, dilation=1, bias=False)
dilation_size_stub_constructor = partial(MyConvStub, input_channels=128, output_channels=128, kernel_size=(3, 3), num_groups=1, stride=1, bias=False)
groups_size_stub_constructor = partial(MyConvStub, input_channels=128, output_channels=256, kernel_size=(3, 3), stride=1, dilation=1, bias=False)


ATOL = 1e-3

class ConvTests(unittest.TestCase):
    """A suite of test that performs numerical checking on ConvStub"""
    def test_square_kernels(self):
        kernel_sizes = [(k, k) for k in range(1, 8)]
        for kernel_size in kernel_sizes:
            # create a conv stub
            conv = kernel_size_stub_constructor(kernel_size=kernel_size)
            random_input = torch.randn(4, 128, 64, 64)

            output = conv.forward(random_input)
            test_output = F.conv2d(
                input=random_input,
                weight=conv.weight,
                groups=conv.groups,
                stride=conv.stride,
                dilation=conv.dilation,
                padding=0,
            )

            self.assertTrue(torch.allclose(output, test_output, atol=ATOL))

    def test_horizontal_kernel(self):
        kernel_sizes = [(k, k + 2) for k in range(1, 8)]
        for kernel_size in kernel_sizes:
            # create a conv stub
            conv = kernel_size_stub_constructor(kernel_size=kernel_size)
            random_input = torch.randn(4, 128, 64, 64)

            output = conv.forward(random_input)
            test_output = F.conv2d(
                input=random_input,
                weight=conv.weight,
                groups=conv.groups,
                stride=conv.stride,
                dilation=conv.dilation,
                padding=0,
            )

            self.assertTrue(torch.allclose(output, test_output, atol=ATOL))

    def test_vertical_kernel(self):
        kernel_sizes = [(k + 2, k) for k in range(1, 8)]
        for kernel_size in kernel_sizes:
            # create a conv stub
            conv = kernel_size_stub_constructor(kernel_size=kernel_size)
            random_input = torch.randn(4, 128, 64, 64)

            output = conv.forward(random_input)
            test_output = F.conv2d(
                input=random_input,
                weight=conv.weight,
                groups=conv.groups,
                stride=conv.stride,
                dilation=conv.dilation,
                padding=0,
            )

            self.assertTrue(torch.allclose(output, test_output, atol=ATOL))

    def test_increasing_filter_sizes(self):
        input_sizes = [(2 ** i) for i in range(0, 8)]
        output_sizes = [(3 ** i) for i in range(1, 6)]
        for input_size, output_size in zip(input_sizes, output_sizes):
            # create a conv stub
            conv = filter_size_stub_constructor(input_channels=input_size, output_channels=output_size)
            random_input = torch.randn(4, input_size, 64, 64)

            output = conv.forward(random_input)
            test_output = F.conv2d(
                input=random_input,
                weight=conv.weight,
                groups=conv.groups,
                stride=conv.stride,
                dilation=conv.dilation,
                padding=0,
            )

            self.assertTrue(torch.allclose(output, test_output, atol=ATOL))

    def test_decreasing_filter_sizes(self):
        output_sizes = [(2 ** i) for i in range(0, 8)]
        input_sizes = [(3 ** i) for i in range(1, 6)]
        for input_size, output_size in zip(input_sizes, output_sizes):
            # create a conv stub
            conv = filter_size_stub_constructor(input_channels=input_size, output_channels=output_size)
            random_input = torch.randn(4, input_size, 64, 64)

            output = conv.forward(random_input)
            test_output = F.conv2d(
                input=random_input,
                weight=conv.weight,
                groups=conv.groups,
                stride=conv.stride,
                dilation=conv.dilation,
                padding=0,
            )

            self.assertTrue(torch.allclose(output, test_output, atol=ATOL))

    def test_same_number_of_filters(self):
        input_sizes = [(2 ** i) for i in range(0, 8)]
        output_sizes = [(2 ** i) for i in range(0, 8)]
        for input_size, output_size in zip(input_sizes, output_sizes):
            # create a conv stub
            conv = filter_size_stub_constructor(input_channels=input_size, output_channels=output_size)
            random_input = torch.randn(4, input_size, 64, 64)

            output = conv.forward(random_input)
            test_output = F.conv2d(
                input=random_input,
                weight=conv.weight,
                groups=conv.groups,
                stride=conv.stride,
                dilation=conv.dilation,
                padding=0,
            )

            self.assertTrue(torch.allclose(output, test_output, atol=ATOL))

    def test_stride(self):
        strides = list(range(1, 4))
        for stride in strides:
            # create a conv stub
            conv = stride_size_stub_constructor(stride=stride)
            random_input = torch.randn(4, 128, 64, 64)

            output = conv.forward(random_input)
            test_output = F.conv2d(
                input=random_input,
                weight=conv.weight,
                groups=conv.groups,
                stride=conv.stride,
                dilation=conv.dilation,
                padding=0
            )

            self.assertTrue(torch.allclose(output, test_output, atol=ATOL))

    def test_dilation(self):
        dilations = list(range(1, 4))
        for dilation in dilations:
            # create a conv stub
            conv = dilation_size_stub_constructor(dilation=dilation)
            random_input = torch.randn(4, 128, 64, 64)

            print(dilation)
            output = conv.forward(random_input)
            test_output = F.conv2d(
                input=random_input,
                weight=conv.weight,
                groups=conv.groups,
                stride=conv.stride,
                dilation=conv.dilation,
                padding=0
            )

            self.assertTrue(torch.allclose(output, test_output, atol=ATOL))

    def test_grouping(self):
        groups = [4, 8, 16, 32, 64]
        for group in groups:
            # create a conv stub
            conv = groups_size_stub_constructor(num_groups=group)
            random_input = torch.randn(4, 128, 64, 64)

            output = conv.forward(random_input)
            test_output = F.conv2d(
                input=random_input,
                weight=conv.weight,
                groups=conv.groups,
                stride=conv.stride,
                dilation=conv.dilation,
                padding=0
            )

            self.assertTrue(torch.allclose(output, test_output, atol=ATOL))

    def test_biased_conv(self):
        input_sizes = [(2 ** i) for i in range(0, 8)]
        output_sizes = [(3 ** i) for i in range(1, 6)]
        for input_size, output_size in zip(input_sizes, output_sizes):
            # create a conv stub
            conv = biased_filter_size_stub_constructor(input_channels=input_size, output_channels=output_size)
            random_input = torch.randn(4, input_size, 64, 64)

            output = conv.forward(random_input)
            test_output = F.conv2d(
                input=random_input,
                weight=conv.weight,
                groups=conv.groups,
                stride=conv.stride,
                dilation=conv.dilation,
                bias=conv.bias,
                padding=0,
            )

            self.assertTrue(torch.allclose(output, test_output, atol=ATOL))

    def test_zero_biased_conv(self):
        input_sizes = [(2 ** i) for i in range(0, 8)]
        output_sizes = [(3 ** i) for i in range(1, 6)]
        for input_size, output_size in zip(input_sizes, output_sizes):
            # create a conv stub
            conv = filter_size_stub_constructor(input_channels=input_size, output_channels=output_size)
            random_input = torch.randn(4, input_size, 64, 64)

            output = conv.forward(random_input)
            test_output = F.conv2d(
                input=random_input,
                weight=conv.weight,
                groups=conv.groups,
                stride=conv.stride,
                dilation=conv.dilation,
                bias=torch.zeros(output_size),
                padding=0,
            )

            self.assertTrue(torch.allclose(output, test_output, atol=ATOL))




class FilterTest(unittest.TestCase):
    """A suite of tests that numerically checks the filtering operation"""

    class Filter2D(nn.Module):
        def __init__(self, channels: int, kernel: torch.Tensor):
            super().__init__()
            self.register_buffer('filter', kernel[None, None, :, :].repeat(channels, 1, 1, 1))

        def forward(self, x: torch.Tensor) -> torch.Tensor:
            b, c, h, w = x.shape
            return F.conv2d(
                x,
                self.filter,
                stride=1,
                padding=0,
                groups=c,
            )

    def test_filter(self):
        blur_sizes = list(range(1, 7))
        for blur_size in blur_sizes:
            # create a filter stub
            blur_kernel = get_blur_kernel(blur_size)
            conv = MyFilterStub(filter=blur_kernel, input_channels=128)
            test_conv = self.Filter2D(128, blur_kernel)
            random_input = torch.randn(4, 128, 64, 64)
            output = conv.forward(random_input)
            test_output = test_conv.forward(random_input)

            self.assertTrue(torch.allclose(output, test_output, atol=ATOL))



In [6]:
convTests = ConvTests()

In [7]:
convTests.test_square_kernels()

In [8]:
convTests.test_horizontal_kernel()

In [9]:
convTests.test_vertical_kernel()

In [10]:
convTests.test_increasing_filter_sizes()

In [11]:
convTests.test_decreasing_filter_sizes()

In [12]:
convTests.test_same_number_of_filters()

In [13]:
convTests.test_stride()

In [14]:
convTests.test_dilation()

1
2


AssertionError: False is not true

In [15]:
convTests.test_biased_conv()

In [16]:
convTests.test_zero_biased_conv()

In [17]:
convTests.test_grouping()

RuntimeError: The size of tensor a (128) must match the size of tensor b (32) at non-singleton dimension 0

In [18]:
filterTest = FilterTest()

In [19]:
filterTest.test_filter()